# File groups restores

How do we restore file group backups - It can be done in SIMPLE recovery mode but **only** for read-only filegroups

> In a file restore, the goal is to restore one or more damaged files without restoring the whole database. Under the simple recovery model, file backups are supported only for read-only files. **The primary filegroup and read/write secondary filegroups are always restored together, by restoring a database or partial backup.**

https://docs.microsoft.com/en-us/sql/relational-databases/backup-restore/file-restores-simple-recovery-model?view=sql-server-ver15

So do the load, set the file groups(s) to read only and then have a process to restore them in the same way

But

You need to be to only connection to make the `ALTER DATABASE MYDB MODIFY FILEGROUP FGName READ_ONLY` code which means that you need to run

```
ALTER DATABASE MyDB SET SINGLE_USER;  
ALTER DATABASE MYDB MODIFY FILEGROUP FGName READ_ONLY  
ALTER DATABASE MyDB SET MULTI_USER;
```
after the load and `READ_WRITE` before each load
Which is adding more complexity. So the decision is based on the answer to

> How important are filegroup restores compared to additional administration or more complex processes

and

> Will we know before making the file group read-write again that we have a mistake and need to restore?

Also, setting the mart read only after a load could be a good thing??

## Database

In [ ]:
CREATE DATABASE FileGroup_restores
GO


Add file groups and files

In [ ]:
ALTER DATABASE FileGroup_restores 
ADD FILEGROUP FGTestFG1
ALTER DATABASE FileGroup_restores 
ADD FILEGROUP FGTestFG2

ALTER DATABASE FileGroup_restores 
ADD FILE 
(NAME = FileGroup_number1, FILENAME = 'FileGroup_FileGroup1.ndf') 
TO FILEGROUP FGTestFG1

ALTER DATABASE FileGroup_restores 
ADD FILE 
(NAME = FileGroup_number2, FILENAME = 'P:\DATA\GBQIS1SDA01ODFileGroup_FileGroup2.ndf') 
TO FILEGROUP FGTestFG2

Add a table on each file group and fill with data

In [ ]:
USE FileGroup_restores 
GO
CREATE TABLE [dbo].[StuffAndJunk](
	[StuffHere] [bigint] NULL,
	[JunkHere] [bigint] NULL,
	[DuffStuffHere] [bigint] NULL
) ON [FGTestFG1]


 ; WITH
  L0   AS (SELECT c FROM (SELECT 1 UNION ALL SELECT 1) AS D(c)), -- 2^1
  L1   AS (SELECT 1 AS c FROM L0 AS A CROSS JOIN L0 AS B),       -- 2^2
  L2   AS (SELECT 1 AS c FROM L1 AS A CROSS JOIN L1 AS B),       -- 2^4
  L3   AS (SELECT 1 AS c FROM L2 AS A CROSS JOIN L2 AS B),       -- 2^8
  L4   AS (SELECT 1 AS c FROM L3 AS A CROSS JOIN L3 AS B),       -- 2^16
  L5   AS (SELECT 1 AS c FROM L4 AS A CROSS JOIN L4 AS B),       -- 2^32
  Nums AS (SELECT ROW_NUMBER() OVER(ORDER BY (SELECT NULL)) AS k FROM L5)

INSERT INTO [dbo].[StuffAndJunk] WITH (TABLOCK)  -- to reduce logging in simple mode_
select k as StuffHere , k/2 as JunkHere,  k/4 as DuffStuffHere
from nums
where k <= 1000000
GO

USE FileGroup_restores 
GO
CREATE TABLE [dbo].[NewStuff](
	[StuffHere] [bigint] NULL,
	[JunkHere] [bigint] NULL,
	[DuffStuffHere] [bigint] NULL
) ON [FGTestFG2]


 ; WITH
  L0   AS (SELECT c FROM (SELECT 1 UNION ALL SELECT 1) AS D(c)), -- 2^1
  L1   AS (SELECT 1 AS c FROM L0 AS A CROSS JOIN L0 AS B),       -- 2^2
  L2   AS (SELECT 1 AS c FROM L1 AS A CROSS JOIN L1 AS B),       -- 2^4
  L3   AS (SELECT 1 AS c FROM L2 AS A CROSS JOIN L2 AS B),       -- 2^8
  L4   AS (SELECT 1 AS c FROM L3 AS A CROSS JOIN L3 AS B),       -- 2^16
  L5   AS (SELECT 1 AS c FROM L4 AS A CROSS JOIN L4 AS B),       -- 2^32
  Nums AS (SELECT ROW_NUMBER() OVER(ORDER BY (SELECT NULL)) AS k FROM L5)

INSERT INTO [dbo].[NewStuff] WITH (TABLOCK)  -- to reduce logging in simple mode_
select k as StuffHere , k/2 as JunkHere,  k/4 as DuffStuffHere
from nums
where k <= 1000000
GO

Take a backups

In [ ]:
Use Master
GO
ALTER DATABASE FileGroup_restores  SET SINGLE_USER;  
ALTER DATABASE FileGroup_restores  MODIFY FILEGROUP FGTestFG2 READ_ONLY  
ALTER DATABASE FileGroup_restores  SET MULTI_USER;

BACKUP DATABASE FileGroup_restores  Read_Write_FileGroups 
TO DISK ='FileGroup_restores _RW.bak'

BACKUP DATABASE FileGroup_restores   
   FILEGROUP = 'FGTestFG2' 
   TO DISK = 'FileGroup_restores _FG2.bak';  
GO  


Delete from table (simultate duff load)

In [ ]:
USE FileGroup_restores 
GO
ALTER DATABASE FileGroup_restores  SET SINGLE_USER;  
ALTER DATABASE FileGroup_restores  MODIFY FILEGROUP FGTestFG2 READ_WRITE 
ALTER DATABASE FileGroup_restores  SET MULTI_USER;
TRUNCATE TABLE [dbo].[NewStuff]
ALTER DATABASE FileGroup_restores  SET SINGLE_USER;  
ALTER DATABASE FileGroup_restores  MODIFY FILEGROUP FGTestFG2 READ_ONLY  
ALTER DATABASE FileGroup_restores  SET MULTI_USER;

Check

In [ ]:
USE FileGroup_restores
GO
SELECT TOP 1 * FROM [dbo].[StuffAndJunk]
SELECT TOP 1 * FROM [dbo].[NewStuff]

Oh no - our load is screwed (we have no data in NewStuff) - assume StuffAndJunk has changed.

## Restore all readwrite parts of database

In [ ]:
USE master
ALTER DATABASE FileGroup_restores SET SINGLE_USER;  
RESTORE DATABASE FileGroup_restores READ_WRITE_FILEGROUPS FROM Disk ='FileGroup_restores_RW.bak'
WITH PARTIAL, RECOVERY  


Can read from FG1 but not FG2

In [ ]:
USE FileGroup_restores
GO
SELECT TOP 1 * FROM [dbo].[StuffAndJunk]


In [ ]:
USE FileGroup_restores
GO
SELECT TOP 1 * FROM [dbo].[NewStuff]

Restore FG2

In [ ]:
USE master
RESTORE DATABASE FileGroup_restores 
FROM Disk ='FileGroup_restores_FG2.bak' 
WITH RECOVERY

Check

In [ ]:
USE FileGroup_restores
GO
SELECT TOP 1 * FROM [dbo].[StuffAndJunk]
SELECT TOP 1 * FROM [dbo].[NewStuff]

# Clean Up

In [ ]:
USE [master]
GO
ALTER DATABASE [FileGroup_restores] SET SINGLE_USER WITH ROLLBACK IMMEDIATE
DROP DATABASE FileGroup_restores

In [ ]:
DECLARE @SQL NVARCHAR(500)
DECLARE @PowerShell NVarchAr(500)

SET @sql =  N' DEL FileGroup_restores_RW.bak ; FileGroup_restores_FG2.bak'
-- PRINT @SQL
EXEC master..xp_cmdshell @SQL